# Análise Preditiva de Suicídios

## Descrição do dataset
Os dados foram coletados em forma de questionário através de redes sociais e emails. Alguns direcionados a grupos específicos de apoio (contra depressão). As questões abordadas foram baseadas nesta [pesquisa](https://www.kaggle.com/kingburrito666/the-demographic-rforeveralone-dataset). <br>

<b>Descrição das variáveis:</b><br>
* Q1: "Qual a sua idade? <br>
* Q2: "Qual gênero você mais se identifica?" <br>
* Q3: "Qual sua orientação  sexual?" <br>
* Q4: "Qual o seu nível de renda anual?" <br>
* Q5: "Qual sua etnicidade?" <br>
* Q6: "Como você descreve sua massa corporal?" <br>
* Q7: "Você é virgem?" <br>
* Q8: "Você pagaria para ter uma relação sexual?" <br>
* Q9: "Quantos amigos você tem na vida real?" <br>
* Q10: "Você possui ansiedade/fobia social?" <br>
* Q11: "Você possui algum tipo de diagnóstico de transtorno mental?" <br>
* Q12: "Que tipo de ajuda você quer? <br>
* Q13: "Você já tentou se suicidar?" <br>
* Q14: "Qual seu status empregatício? Atualmente você é..." <br>
* Q15: "Qual seu cargo atual?" <br>
* Q16: "Qual seu nível de educação?" <br>
* Q17: "O que você fez para se sentir melhor consigo mesmo?" <br>
* Q18: "Com quem você mora?" <br>
* Q19: "Seus pais são divorciados?" <br>


In [179]:
import pandas as pd

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 100)

In [180]:
survey_data = pd.read_csv('survey_data.csv')

In [181]:
df = survey_data.iloc[:, 1:20]
df.columns = ['Q{}'.format(x+1) for x in range(0, 19)]
df.isnull().sum()

Q1     0  
Q2     0  
Q3     0  
Q4     6  
Q5     0  
Q6     0  
Q7     0  
Q8     0  
Q9     0  
Q10    0  
Q11    84 
Q12    0  
Q13    0  
Q14    0  
Q15    111
Q16    0  
Q17    0  
Q18    0  
Q19    0  
dtype: int64

## 1. Pré-processamento

In [182]:
def check_values_column(series):
    return pd.DataFrame(series.str.strip().unique()).sort_values(by=0)

### 1.1 Variável Q4

In [183]:
df[df['Q4'].isna()].shape

(6, 19)

In [184]:
df['Q4'].value_counts()

R$1,00 - R$9,999         101
R$10,000 - R$19,999      42 
R$20,000 - R$29,999      29 
R$50,000 - R$74,999      19 
R$200,000 ou mais        12 
R$30,000 - R$39,999      11 
R$75,000 - R$99,999      7  
R$40,000 - R$49,999      7  
R$125,000 - R$149,999    4  
R$100,000 - R$124,999    3  
R$175,000 - R$199,999    2  
R$150,000 - R$174,999    1  
Name: Q4, dtype: int64

#### 1.1.1 Substituindo valores pela moda

Baseado na análise, os registros são referente a estudantes e/ou desempregados

In [185]:
mode = df['Q4'].value_counts().index[0]

In [186]:
df['Q4'].fillna(mode, inplace=True)

### 1.2 Variável Q11
Esta variável envolve multipla seleção e digitação em campo livre.<br>
Os passos para tratamento serão:<br>
* Atribuir valor caso não tenha sido informado;
* Sobrescrever valores redundantes (digitados);
* Converter multipla seleção em colunas separadas;
* Converter as classes separadas para binário;
* Classificar valores digitados manualmente em categorias existentes e/ou criando novas (se necessário);

In [187]:
df[df['Q11'].isna()].shape

(84, 19)

In [188]:
df['Q11'].fillna('Sem resposta', inplace=True)

In [189]:
check_value_columns(df['Q11'])

,0
21,Ansiedade de leve
12,"Apenas períodos de ansiedade + depressão , mas nao transtornos que me acompanharam durante a vida"
28,Nenhum
38,Nenhum (que eu saiba).
33,Nenhum diagnóstico
23,Não
39,Não possuo diagnóstico
30,Não tenho como responder pois evito procurar ajuda
45,"Não tenho nada diagnosticado, mas me sinto constantemente sobrecarregado e extremamente estressado. Pelo que leio, isso se chama Sindrome de Burn Out."
35,Não tenho transtorno mental


#### 1.2.1 Agrupando/sobrescrevendo valores redundantes

In [190]:
nenhum = 'Nenhum'

df.loc[df.Q11.str.lower().str.contains('nenhum'), 'Q11'] = nenhum
df.loc[df.Q11.str.lower().str.contains('não'), 'Q11'] = nenhum
df.loc[df.Q11.str.lower().str.contains('sem diagnóstico médico'), 'Q11'] = nenhum

outros = 'Outros'
df.loc[df.Q11.str.lower().str.contains('ansiedade de leve'), 'Q11'] = outros
df.loc[df.Q11.str.lower().str.contains('apenas períodos de ansiedade'), 'Q11'] = outros
df.loc[df.Q11.str.lower().str.contains('problemas psicológicos após lesão corporal'), 'Q11'] = outros
df.loc[df.Q11.str.lower().str.contains('tdah'), 'Q11'] = outros

df.Q11 = df.Q11.str.strip().str.replace('toc', 'TOC', regex=True)

In [191]:
check_values_column(df['Q11'])

,0
21,Nenhum
11,Outros
1,Sem resposta
0,"Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...)"
29,"Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...);Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)"
19,"Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...);Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)"
15,"Transtornos Alimentares (Anorexia, Bulimia, Transtorno de Ruminação, etc...)"
34,"Transtornos Alimentares (Anorexia, Bulimia, Transtorno de Ruminação, etc...);Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...);Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)"
18,"Transtornos Alimentares (Anorexia, Bulimia, Transtorno de Ruminação, etc...);Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)"
26,"Transtornos Alimentares (Anorexia, Bulimia, Transtorno de Ruminação, etc...);Transtornos de Ansiedade (Fobia Específica, Transtorno de Ansiedade Generalizada, Transtorno de Pânico, etc...)"


#### 1.2.2 Transformando variáveis categóricas em binário
Criando colunas binárias separadas para cada tipo de transtorno.

In [197]:
df['Q11'].str.strip().str.split(';', expand=True)

,0,1,2,3,4,5
0,"Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...)",None,None,None,None,None
1,Sem resposta,None,None,None,None,None
2,"Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)",None,None,None,None,None
3,"Transtornos de Ansiedade (Fobia Específica, Transtorno de Ansiedade Generalizada, Transtorno de Pânico, etc...)",None,None,None,None,None
4,"Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...)",None,None,None,None,None
5,"Transtornos de Ansiedade (Fobia Específica, Transtorno de Ansiedade Generalizada, Transtorno de Pânico, etc...)",None,None,None,None,None
6,Sem resposta,None,None,None,None,None
7,Sem resposta,None,None,None,None,None
8,Sem resposta,None,None,None,None,None
9,"Transtornos de Ansiedade (Fobia Específica, Transtorno de Ansiedade Generalizada, Transtorno de Pânico, etc...)","Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...)","Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)","Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)","Transtornos Relacionados a Trauma e a Estressores (Transtorno de Apego Reativo, Transtorno de Interação Social Desinibida, Transtorno de Estresse Pós-traumático, Transtorno de Estresse Agudo, etc..)",None


In [196]:
df = pd.concat([df, pd.get_dummies(df['Q11'].str.strip().str.split(';', expand=True))], axis=1)

In [193]:
df.columns

Index(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', '0_Nenhum',
       '0_Outros', '0_Sem resposta',
       '0_Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...)',
       '0_Transtornos Alimentares  (Anorexia, Bulimia, Transtorno de Ruminação, etc...)',
       '0_Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)',
       '0_Transtornos Relacionados a Trauma e a Estressores (Transtorno de Apego Reativo, Transtorno de Interação Social Desinibida, Transtorno de Estresse Pós-traumático, Transtorno de Estresse Agudo, etc..)',
       '0_Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)',
       '0_Transtornos de Ansiedade (Fobia Específica, Transtorno de Ansiedade Generalizada, Transtorno de Pânico, etc